In [2]:
# Forge Machine Learning API: Classification
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit import Aer, IBMQ
import random
from scipy.spatial import distance
shotNum=2048
plus_minus=[-1,1]
verbose=1
# Hardware run flag
flag_hw = 0
for i in range(10):
    #####################
    # make random inputs
    #X=np.array([[1,-1,-1,-1],[1, -1, -1, 1],[-1,-1,1,-1]])
    #T=np.array([[-1, -1, 1, 1]])
    X= np.random.choice(plus_minus, size=(3, 4))
    T= np.random.choice(plus_minus, size=(1, 4))
    print(X)
    print(T)
    #######################
    # Compute the distances classically for verification
    dist = []
    min_list=[]
    for i in range(len(X)):
        dist.append(distance.euclidean(X[i],T))
    min_list=[i for i in range(len(dist)) if dist[i]==min(dist)]
    print("Minimum distance:",min(dist)**2/16)
    print("Label:",min_list)
    
    # initialize the minimum distance and indices
    
    min_distance_sim=1
    min_index_sim=-1
    min_distance_hw=1
    min_index_hw=-1
    # Run the quantum circuit
    
    for i in range(len(X)):
        # Create a Quantum Register with 6 qubits.
        q = QuantumRegister(6)
        # Create a Classical Register with 1 bit.
        c = ClassicalRegister(1)
        # Create a Quantum Circuit
        qc = QuantumCircuit(q, c)
        # Create the state |01>+|10> on the first two qubits
        qc.h(q[0])
        qc.cx(q[0], q[1])
        qc.x(q[0])
        # Create the state |0000>+|0101>+|1010>+|1111> on qubits 2,3,4,5
        qc.h(q[2])
        qc.h(q[3])
        qc.cx(q[2], q[4])
        qc.cx(q[3], q[5])
        if flag_hw==0: qc.barrier()
        # Create conditionally the states X_i and T
        if X[i, 0] == -1:
            qc.z(q[0])
        if T[0, 0] == -1:
            qc.z(q[1])
        if X[i,0]*X[i,1]*X[i,2]*X[i,3]==-1:
            qc.h(q[3])
            qc.ccx(q[0],q[2],q[3])
            qc.h(q[3])
        if T[0,0]*T[0,1]*T[0,2]*T[0,3]==-1:
            qc.h(q[5])
            qc.ccx(q[1],q[4],q[5])
            qc.h(q[5])
        if X[i,0]*X[i,2]==-1:
            qc.cz(q[0],q[2])
        if T[0, 0]*T[0,2]==-1:
            qc.cz(q[1],q[4])
        if X[i, 0]*X[i,1]==-1:
            qc.cz(q[0],q[3])
        if T[0, 0]*T[0,1]==-1:
            qc.cz(q[1],q[5])
        if flag_hw==0: qc.barrier()
        # Unentangle first and second qubit
        qc.x(q[0])
        qc.cx(q[0], q[1])
        qc.h(q[0])
        # Measure the first qubit
        qc.measure(q[0],c[0])
        # Print the circuit
        #qc.draw(output='mpl')
        if verbose==1: print(qc)
        #
        # Run in simulator.
        #
        backend_sim = Aer.get_backend('qasm_simulator')
        # Compile and run the Quantum circuit on a simulator backend
        job_sim = execute(qc, backend_sim,shots=shotNum)
        result_sim = job_sim.result()
        # Show the results
        #print(result_sim.get_counts(qc))
        if '0' in result_sim.get_counts(qc):
            #print((1-result_sim.get_counts(qc)['0']/shotNum))
            if (1-result_sim.get_counts(qc)['0']/shotNum) <= min_distance_sim:
                min_distance_sim=(1-result_sim.get_counts(qc)['0']/shotNum)
                min_index_sim=i
        #
        # Run in hardware.
        #
        if flag_hw == 1:
            hardware_backend = IBMQ.get_backend('ibmq_20_tokyo')
            # Compile and run the Quantum circuit on a simulator backend
            job_hardware = execute(qc, hardware_backend, shots=shotNum)
            result_hardware = job_hardware.result()
            # Show the results
            #print(result_hardware.get_counts(qc))
            if '0' in result_hardware.get_counts(qc):
                #print((1-result_hardware.get_counts(qc)['0']/shotNum))
                if (1-result_hardware.get_counts(qc)['0']/shotNum) <= min_distance_hw:
                    min_distance_hw=(1-result_hardware.get_counts(qc)['0']/shotNum)
                    min_index_hw=i
# printing the results
    if flag_hw == 0: 
        print("Minimum distance (sim):", min_distance_sim)
        print("Label (sim):", min_index_sim)
    else:
        print("Minimum distance (hw):", min_distance_hw)
        print("Label (hw):",min_index_hw)
# Verification
    if flag_hw == 0: 
        if min_index_sim in min_list:
            print("Correct Simulation")
        else: 
            print("Wrong Simulation")
    else:
        if min_index_hw in min_list:
            print("Correct Hardware Execution")
        else:
            print("Wrong Hardware Execution")

[[ 1  1  1  1]
 [-1  1 -1 -1]
 [-1  1  1  1]]
[[ 1 -1 -1 -1]]
Minimum distance: 0.5000000000000001
Label: [1]
                              ┌───┐     ┌───┐ ░                       ░ ┌───┐»
q30_0: |0>────────────────────┤ H ├──■──┤ X ├─░───────────────────────░─┤ X ├»
                              └───┘┌─┴─┐└───┘ ░                       ░ └───┘»
q30_1: |0>─────────────────────────┤ X ├──────░────────■───■───────■──░──────»
                    ┌───┐          └───┘      ░        │   │       │  ░      »
q30_2: |0>──────────┤ H ├──■──────────────────░────────┼───┼───────┼──░──────»
          ┌───┐     └───┘  │                  ░        │   │       │  ░      »
q30_3: |0>┤ H ├──■─────────┼──────────────────░────────┼───┼───────┼──░──────»
          └───┘  │       ┌─┴─┐                ░        │   │       │  ░      »
q30_4: |0>───────┼───────┤ X ├────────────────░────────■───■───────┼──░──────»
               ┌─┴─┐     └───┘                ░ ┌───┐┌─┴─┐   ┌───┐ │  ░      »
q30_5: |0>─────┤ X ├─

                              ┌───┐     ┌───┐ ░             ░ ┌───┐     ┌───┐»
q40_0: |0>────────────────────┤ H ├──■──┤ X ├─░──■──────────░─┤ X ├──■──┤ H ├»
                              └───┘┌─┴─┐└───┘ ░  │ ┌───┐    ░ └───┘┌─┴─┐└───┘»
q40_1: |0>─────────────────────────┤ X ├──────░──┼─┤ Z ├─■──░──────┤ X ├─────»
                    ┌───┐          └───┘      ░  │ └───┘ │  ░      └───┘     »
q40_2: |0>──────────┤ H ├──■──────────────────░──■───────┼──░────────────────»
          ┌───┐     └───┘  │                  ░          │  ░                »
q40_3: |0>┤ H ├──■─────────┼──────────────────░──────────┼──░────────────────»
          └───┘  │       ┌─┴─┐                ░          │  ░                »
q40_4: |0>───────┼───────┤ X ├────────────────░──────────┼──░────────────────»
               ┌─┴─┐     └───┘                ░          │  ░                »
q40_5: |0>─────┤ X ├──────────────────────────░──────────■──░────────────────»
               └───┘                          ░     

                              ┌───┐     ┌───┐ ░                             »
q50_0: |0>────────────────────┤ H ├──■──┤ X ├─░─────────────■────────■──────»
                              └───┘┌─┴─┐└───┘ ░             │        │ ┌───┐»
q50_1: |0>─────────────────────────┤ X ├──────░─────────────┼────────┼─┤ Z ├»
                    ┌───┐          └───┘      ░             │        │ └───┘»
q50_2: |0>──────────┤ H ├──■──────────────────░─────────────■────────┼──────»
          ┌───┐     └───┘  │                  ░      ┌───┐┌─┴─┐┌───┐ │      »
q50_3: |0>┤ H ├──■─────────┼──────────────────░──────┤ H ├┤ X ├┤ H ├─■──────»
          └───┘  │       ┌─┴─┐                ░      └───┘└───┘└───┘        »
q50_4: |0>───────┼───────┤ X ├────────────────░─────────────────────────────»
               ┌─┴─┐     └───┘                ░ ┌───┐                       »
q50_5: |0>─────┤ X ├──────────────────────────░─┤ H ├───────────────────────»
               └───┘                          ░ └───┘           

Minimum distance (sim): 0.251953125
Label (sim): 2
Correct Simulation
